In [7]:
# import yaml
# import random
# import argparse
# from collections import namedtuple
# from tqdm.notebook import tqdm
# from pprint import pprint
# from my_utils.utils import print_dict
from pathlib import Path
# import shutil
import numpy as np
# import pandas as pd

In [8]:
from rtal.data.detector import Detector

In [11]:
dataset_folder = Path('demo_dataset')
fnames = sorted(list(dataset_folder.glob('*npz')))

[PosixPath('demo_dataset/sample_0.npz'), PosixPath('demo_dataset/sample_1.npz'), PosixPath('demo_dataset/sample_2.npz'), PosixPath('demo_dataset/sample_3.npz')]


### Sanity check 1
data dimension

In [12]:
fname = fnames[0]
with np.load(fname) as handle:
    detector_start     = handle['detector_start']
    detector_curr      = handle['detector_curr']
    readout_start      = handle['readout_start'] 
    readout_curr       = handle['readout_curr'] 
    particle_vertex    = handle['particle_vertex']
    particle_direction = handle['particle_direction']

print(f'detector start:\n{detector_start}\n')
print(f'detector misaligned:\n{np.array2string(detector_curr, sign="+", formatter={"float_kind":lambda x: f"{x:.8f}"})}\n')
print(f'readout shape: {readout_start.shape}\n')
print(f'readout start: \n{np.array2string(readout_start[:, :5], precision=4)}\n')
print(f'readout misaligned: \n{np.array2string(readout_curr[:, :5], precision=4)}\n')
print(f'particle vertex:\n{np.array2string(particle_vertex[:5], precision=4)}\n')
print(f'particle direction:\n{np.array2string(particle_direction[:5], precision=4)}\n')

detector start:
[[ 0. 10.  0. -1.  0.  0.  0.  0.  1.]
 [ 0. 20.  0. -1.  0.  0.  0.  0.  1.]
 [ 0. 30.  0. -1.  0.  0.  0.  0.  1.]]

detector misaligned:
[[0.11245309 10.07322673 -0.10665619 -0.99997772 0.00000000 -0.00667465
  -0.00667465 0.00000000 0.99997772]
 [0.00000000 20.00000000 0.00000000 -0.99520105 0.09780621 0.00297037
  -0.00297037 -0.06053829 0.99816146]
 [0.00000000 30.00000000 0.00000000 -0.99997650 0.00000000 -0.00685613
  -0.00685613 0.00000000 0.99997650]]

readout shape: (3, 105, 2)

readout start: 
[[[ -5.  -5.]
  [  7.  -9.]
  [ 10.  -5.]
  [ 13.  -5.]
  [-10.  -3.]]

 [[ -9. -11.]
  [ 16. -20.]
  [ 22.  -9.]
  [ 27.  -9.]
  [-18.  -6.]]

 [[-14. -17.]
  [ 25. -31.]
  [ 33. -14.]
  [ 40. -13.]
  [-27.  -8.]]]

readout misaligned: 
[[[ -4.  -4.]
  [  8.  -8.]
  [ 12.  -4.]
  [ 14.  -3.]
  [ -9.  -2.]]

 [[ -9. -11.]
  [ 17. -20.]
  [ 22. -10.]
  [ 27.  -9.]
  [-18.  -6.]]

 [[-13. -17.]
  [ 25. -30.]
  [ 34. -14.]
  [ 40. -13.]
  [-27.  -9.]]]

particle vertex:
[

### Sanity Check 2
Get the global intersection points from detector readouts (either normal or misaligned) and compare the ray directions recovered from the readout to the ground-truth ray directions. 

passed!

In [14]:
detectors = []
for param_start, param_curr in zip(detector_start, detector_curr):
    parameter_dict = {'start': param_start, 'curr': param_curr}
    detectors.append(Detector.from_dict(parameter_dict))

In [23]:
particle_direction /= np.linalg.norm(particle_direction, axis=1)[:, np.newaxis]

for state in ['start', 'curr']:

    readout = readout_start if state == 'start' else readout_curr
    global_points = np.stack([detector.to_global(readout, state) 
                              for detector, readout in zip(detectors, readout)])

    recovered_direction = global_points[2] - global_points[1]
    recovered_direction /= np.linalg.norm(recovered_direction, axis=1)[:, np.newaxis]

    angles_in_degree = np.arccos(np.array([np.dot(d1, d2) 
                                           for d1, d2 in zip(particle_direction, recovered_direction)])) * 180 / np.pi

    max_difference_angle_in_degree = angles_in_degree.max()
    
    print(f'maximum difference in recovered (from {state} position) '
          f'and ground-truth directions: {max_difference_angle_in_degree:.5f}\u00B0')

maximum difference in recovered (from start position) and ground-truth directions: 0.62116°
maximum difference in recovered (from curr position) and ground-truth directions: 0.63067°
